In [ ]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import re

In [ ]:
def get_chapter(chapterNum, chapterName):
    print(chapterName)
    url = f"https://founders.org/library/1689-confession/{chapterName}/"
    res = requests.get(url)
    soup = BeautifulSoup(res.text)

    ps = soup.find(id="main").find_all('p')
    sections = [re.split("<sup>.</sup>", ps[i].decode_contents()) for i in range(len(ps))]
    sections = [s for s in sections if s != ['\xa0']]

    dfs = []
    for i,section in enumerate(sections):        
        dfs.append(pd.DataFrame({'section': section}))
    df = pd.concat(dfs)    
    df = df[df.section != ""].reset_index(drop=True).rename(columns={'section':'line'})

    def get_paragraph_number(line):
        try:
            if line[:2] in [f'{x}.' for x in range(1,10)]:
                return int(line[:1])
            elif line[:3] in [f'{x}.' for x in range(10,100)]:
                return int(line[:2])            
        except:
            pass

    df['paragraph'] = df.line.apply(lambda x: get_paragraph_number(x)).interpolate(method="pad")
    df['paragraph'] = df.paragraph.astype(int)

    df['line'] = df.line.apply(lambda x: x.replace('<br>','').replace('<sup>',' ').replace('\n','').replace('<br/>','')\
        .replace('/>','').replace('</sup>',' ').replace('<em>','*').replace('</em>','').strip())

    df['chapter'] = chapterNum

    return df[['chapter', 'paragraph', 'line']]

In [ ]:
chapterNames = [
    "chapter-1-the-holy-scriptures",
    "chapter-2-god-and-the-holy-trinity",
    "chapter-3-gods-decree",
    "chapter-4-creation",
    "chapter-5-divine-providence",
    "chapter-6-the-fall-of-mankind-and-sin-and-its-punishment",
    "chapter-7-gods-covenant",
    "chapter-8-christ-the-mediator",
    "chapter-9-free-will",
    "chapter-10-effectual-calling",
    "chapter-11-justification",
    "chapter-12-adoption",
    "chapter-13-sanctification",
    "chapter-14-saving-faith",
    "chapter-15-repentance-to-life-and-salvation",
    "chapter-16-good-works",
    "chapter-17-the-perseverance-of-the-saints",
    "chapter-18-assurance-of-grace-and-salvation",
    "chapter-19-the-law-of-god",
    "chapter-20-the-gospel-and-the-extent-of-its-grace",
    "chapter-21-christian-liberty-and-liberty-of-conscience",
    "chapter-22-religious-worship-and-the-sabbath-day",
    "chapter-23-lawful-oaths-and-vows",
    "chapter-24-civil-government",
    "chapter-25-marriage",
    "chapter-26-the-church",
    "chapter-27-the-communion-of-saints",
    "chapter-28-baptism-and-the-lords-supper",
    "chapter-29-baptism",
    "chapter-30-the-lords-supper",
    "chapter-31-state-humanity-after-death-resurrection-of-dead",
    "chapter-32-the-last-judgment",
]

In [ ]:
confession = pd.concat([get_chapter(i+1,name) for i,name in enumerate(chapterNames)])

In [ ]:
confession.to_pickle('../data/1689_raw.pkl')